In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import cavsim
from cavsim import Measure
from cavsim.connectors import BaseConnector, Connector
from cavsim.channels import ImportChannel, ExportChannel
from cavsim.components import BaseComponent, Component
from cavsim.solvers import BaseSolver, SimpleSolver
from cavsim.fluids import Fluid

In [3]:
from cavsim.pipes.pipe import Pipe
from cavsim.boundaries.left_boundary_pressure import LeftBoundaryPressure
from cavsim.boundaries.left_boundary_velocity import LeftBoundaryVelocity
from cavsim.boundaries.right_boundary_pressure import RightBoundaryPressure
from cavsim.boundaries.right_boundary_velocity import RightBoundaryVelocity
from cavsim.boundaries.simple_pipe_connector import PipeConnectorSimple
from cavsim.boundaries.zeta_joint import ZetaJoint
from cavsim.boundaries.simple_t_joint import SimpleTJoint
from cavsim.boundaries.simple_right_gasdampener import RightGasBubbleSimple
from cavsim.boundaries.frictionless_gasdampener import FrictionlessGasDampener
from cavsim.boundaries.gasdampener import GasDampener
from cavsim.boundaries.pump_with_valves_model_new import PumpSuctionValve
from cavsim.boundaries.reflectionless_left_boundary_condition import LeftBoundaryReflectionFree
from cavsim.boundaries.reflectionless_right_boundary_condition import RightBoundaryReflectionFree

In [4]:
fluid = Fluid(1000, 1e-3, 2.08e9, 2.3e-3,initial_pressure=128000)

In [5]:
pipe1 = Pipe(1.5, 1.5, 0.002, 2e11, 1e-7, 250)
pipe2 = Pipe(0.056, 4.5, 0.002, 2e11, 1e-7, 250)
pipe4 = Pipe(0.056, 0.5, 0.002, 2e11, 1e-7, 10)
pipe5 = Pipe(0.056, 0.5, 0.002, 2e11, 1e-7, 10)
pipe3 = Pipe(0.025, 28.7, 0.002, 2e11, 1e-7, 250, initial_pressure=20e5)

In [6]:
#lb1 = LeftBoundaryReflectionFree(0, 1.28e5)
lb1 = LeftBoundaryPressure(1.28e5)
pipe1.connect(lb1)

In [7]:
conns = ZetaJoint(1.0)
pipe1.connect(conns)
pipe2.connect(conns)

In [8]:
#conn = SimpleTJoint()
#pipe2.connect(conn)
#pipe4.connect(conn.right2)
#pipe5.connect(conn.right)

In [9]:
#rb1 = RightBoundaryReflectionFree(1.28e5, 0)
#rb1 = RightGasBubbleSimple(128000, 1e-5, 1.0)
#rb1.connect(pipe4)

In [10]:
rb = RightBoundaryPressure(20e5)
rb.connect(pipe3)

In [11]:
pump = PumpSuctionValve(suction_valve_density=7950.0,               # kg/m³
            suction_spring_force0 = 3.438,                  # N
            suction_spring_stiffness = 3438.0,            # N/m
            suction_spring_mass = 0.022,                 # kg
            suction_valve_mass = 0.068,                  # kg
            suction_outer_diameter = 0.040,              # m
            suction_inner_contact_diameter = 0.03644173,      # m
            suction_seat_tilt = 90/180*np.pi,                   # °
            suction_flow_constant_1 = 7.63e-3,             # -
            suction_flow_constant_2 = 6.480,             # -
            suction_friction_factor_a = 107.00,           # -
            suction_friction_factor_b = 74.00,           # -
            suction_friction_factor_c = 1.40,           # -
            suction_friction_factor_d = -2.40,           # -
            suction_factor_k0 = 0.50,                   # -
            suction_factor_k1 = 0.0,                   # -
            suction_factor_k2 = 0.0,                   # -
            suction_max_displacement = 25e-3,                   # m
            suction_outer_contact_diameter = 0.040,      # m
            discharge_valve_density = 7950.0,             # kg/m³
            discharge_spring_force0 = 3.438,             # N
            discharge_spring_stiffness = 3438.0,          # N/m
            discharge_spring_mass = 0.022,               # kg
            discharge_valve_mass = 0.068,                # kg
            discharge_outer_diameter = 0.040,            # m
            discharge_inner_contact_diameter = 0.03644173,    # m
            discharge_seat_tilt = 90/180*np.pi,                 # °
            discharge_flow_constant_1 = 7.63e-3,           # -
            discharge_flow_constant_2 = 6.480,           # -
            discharge_friction_factor_a = 107.00,         # -
            discharge_friction_factor_b = 74.00,         # -
            discharge_friction_factor_c = 1.40,         # -
            discharge_friction_factor_d = -2.40,         # -
            discharge_factor_k0 = 0.50,                 # -
            discharge_factor_k1 = 0.0,                 # -
            discharge_factor_k2 = 0.0,                 # -
            discharge_max_displacement = 25e-3,                 # m
            discharge_outer_contact_diameter = 0.040,    # m
            pump_radius = 0.010/2.0,                         # m
            rpm = 130.0,                                 # U/min
            rratio = 0.175,                              # -
            phi0 = 180,                                # °
            piston_diameter = 0.07,                     # m
            death_volume = 3.442416e-4)                        # m³)

In [12]:
pipe2.connect(pump)
pipe3.connect(pump)

In [13]:
plogs1 = []
plogs2 = []
plogs3 = []
vlogs = []
vlogs1 = []
flogs = []
rblogs = []
relogs = []
dislog = []
springlog = []
contact_pressurelog = []
velocity = []
pumplog = []
pump_speed = []
pump_value = []
flow_log = []
lower_pressure_log = []
contact_pressure_log = []
damping_log = []




def logging(time):
    plogs1.append(pipe1.field_wide_slice('pressure', 0) + 0.0)
    plogs2.append(pipe2.field_wide_slice('pressure', 0) + 0.0)
    plogs3.append(pipe3.field_wide_slice('pressure', 0) + 0.0)
    flogs.append(pipe2.field_wide_slice('friction_steady', 0) + 0.0)
    rblogs.append(rb._velocity[0,1])
    pumplog.append(pump.field_wide_slice('pump_pressure', 0) + 0.0)
    dislog.append(pump.field_wide_slice('suction_displacement', 0) + 0.0)
    relogs.append(pipe2.field_wide_slice('reynolds', 0) + 0.0)
    pump_speed.append(pump.field_wide_slice('discharge_pressure', 1) +0.0)
    pump_value.append(pump.field_wide_slice('discharge_displacement', 1) +0.0)
    springlog.append(pump.field_wide_slice('discharge_valve_velocity', 1) +0.0)
    contact_pressurelog.append(pump.field_wide_slice('suction_contact_pressure_force', 1) +0.0)
    flow_log.append(pump.field_wide_slice('suction_flow_force', 1) +0.0)
    velocity.append(pump.field_wide_slice('suction_valve_velocity', 1) +0.0)
    lower_pressure_log.append(pump.field_wide_slice('discharge_lower_pressure_force', 1) +0.0)
    contact_pressure_log.append(pump.field_wide_slice('discharge_contact_pressure_force', 1) +0.0)
    damping_log.append(pump.field_wide_slice('discharge_upper_pressure_force', 1) +0.0)

In [14]:
solver = SimpleSolver()
solver.fluid = fluid
solver.seeds = [pipe1, pipe2, pipe3]
solver._callback = logging

In [ ]:
solver.solve(1e20, 1.5, 3)

 0:00:00 [                              |  0.01%]  0:54:16  Currently at time   0.000 of   1.500

l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (1.2292131741147046e-05 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x000001F0D8A40DD8>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))
l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (1.4125501757222156e-05 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x000001F0D8A40E48>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))
l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (8.42782927729614e-05 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x000001F0D8A55748>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))


 0:00:17 [                              |  0.81%]  0:35:50  Currently at time   0.012 of   1.500-----------------------------------------
2412.5751008499983
106.94367353401863
0.7717419747525065
3.438
0.0
2513.2741228718346
0.012206086818958822
4.081429838514482e-09
 0:00:17 [                              |  0.81%]  0:35:51  Currently at time   0.012 of   1.500-----------------------------------------
2421.6522756791583
107.10350442058005
0.7717420364110305
3.438
-2.995488247571582e-06
2511.824264363785
0.012230671082441115
2.1434711320071205e-08
 0:00:17 [                              |  0.82%]  0:35:51  Currently at time   0.012 of   1.500-----------------------------------------
2430.7474456226214
107.15598515710032
0.7717423040794867
3.438
-3.453185208298822e-05
2505.530212257498
0.012255255345923408
3.505300521868749e-08
 0:00:17 [                              |  0.82%]  0:35:51  Currently at time   0.012 of   1.500-----------------------------------------
2439.860609148217
107.23

In [ ]:
pfield1 = np.stack(plogs1)
pfield2 = np.stack(plogs2)
pfield3 = np.stack(plogs3)
disfield = np.stack(dislog)
velfield = np.stack(velocity)
pressure = np.stack(pumplog)
pump_velocity = np.stack(pump_speed)
values = np.stack(pump_value)
flow_force = np.stack(flow_log)
spring_force = np.stack(springlog)
lower_pressure_force = np.stack(lower_pressure_log)
contact_pressure_force = np.stack(contact_pressure_log)
damping_force = np.stack(damping_log)

In [ ]:
time = np.linspace(0, 0.7, int(pfield1[:,-2].shape[0]))
#plt.figure(figsize=(16, 10))
fig, ax1 = plt.subplots(figsize=(16, 10))
#plt.subplot(1, 2, 1)
#plt.plot(vfield[:,0])
#plt.plot(1, 2, 2)
val1 = 0
val = -25000
#val1 = 0
#val = -1

#upper_force = upperp_field[val1:val, 0] + gfield[val1:val, 0] + springforce[val1:val, 0]
#lower_force = lowp_field[val1:val, 0] + contact_pressure[val1:val, 0]
#result1 = lower_force - upper_force
#result2 = flow_field[val1:val, 0] - gfield[val1:val, 0] - springforce[val1:val, 0] - dampingfield[val1:val, 0]
#upper_flow = gfield[val1:val, 0] + springforce[val1:val, 0] + dampingfield[val1:val, 0]
#lower_flow = flow_field[val1:val, 0]
#ax1.plot(time[val1:val], upper_flow, color='red')
#ax1.plot(time[val1:val], lower_flow, color='blue')


#plt.subplot(1, 2, 2)
#ax1.plot(result1[val1:val])
#ax1.plot(result2[val1:val])
#ax1.plot(time[val1:val], upperp_field[val1:val, 0], color='red')
#ax1.plot(time[val1:val], lowp_field[val1:val, 0], color='green')
#ax1.plot(time[val1:val], dampingfield[val1:val, 0], color='blue')

#ax1.plot(time[val1:val], contact_pressure[val1:val, 0], color='black')
#ax1.plot(time[val1:val], springforce[val1:val, 0], color='grey')
#ax1.plot(time[val1:val], upper_force, color='red')
#ax1.plot(time[val1:val], lower_force, color='blue')

plt.plot(time[val1:val], pfield3[val1:val, 1], color='red')
plt.plot(time[val1:val], pump_velocity[val1:val, 0], color='black')
plt.plot(time[val1:val], pfield2[val1:val, -2], color='green')
plt.plot(time[val1:val], pressure[val1:val, 0],color='violet')


#ax1.plot(time[val1:val], deltap[val1:val, 0], color='green')

#plt.xlim(time[val1],time[val])
#plt.ylim(0e5, 3e5)

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Pressure', color=color)
#ax1.set_ylim(0e5, 20e5)
#ax1.plot(time[val1:val], pfield1[val1:val, 1], color='red')
#ax1.plot(time[val1:val], pfield2[val1:val, -2], color='green')
ax1.tick_params(axis='y', labelcolor=color)


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.plot(time[val1:val], disfield[val1:val, 0]*1000, color=color)  # we already handled the x-label with ax1
ax2.plot(time[val1:val], values[val1:val, 0]*1000, color = 'black')
#ax2.plot(time[val1:val], contact_pressure_force[val1:val, 0], color = 'green')
#ax2.plot(time[val1:val], spring_force[val1:val, 0], color = 'green')
#ax2.plot(time[val1:val], contact_pressure_force[val1:val, 0]+lower_pressure_force[val1:val, 0], color = 'black')
#ax2.plot(time[val1:val], flow_force[val1:val, 0]-spring_force[val1:val, 0], color = 'red')
#ax2.plot(time[val1:val], zeta[val1:val, 0], color='green')
#ax2.plot(time[val1:val], flow_field[val1:val, 0], color='black')
#ax2.tick_params(axis='y', labelcolor=color)

#plt.subplot(1, 2, 1)
#plt.plot(time[val1:val], disfield[val1:val, 0]*1000, color='black')
#plt.plot(time[val1:val], pump(time[val1:val]))
#plt.plot(time[val1:val], velfield[val1:val, 0], color='blue')
#plt.plot(time[val1:val], pfield2[val1:val, 1], color='red')
#pltplot(time[val1:val], pfield1[val1:val, -2], color='green')
#plt.ylim(-2e3, 2e3)
fig.tight_layout()
plt.show()

# print(pump._cases[val1:val])

In [ ]:
print(pump._cases[val1:val])
print(pressure[val1:val, 0])

#print(np.sum(pump_velocity[val1:val, 0]))

In [ ]:
from scipy import integrate
def contact_pressure(suction_outer_contact_radius, suction_radius,
                     suction_inner_contact_radius, upper_pressure, lower_pressure,
                     density, viscosity, velocity, suction_seat_tilt, displacement):
    pk = (lower_pressure * ((suction_outer_contact_radius / suction_radius - 1)
                                        / (suction_outer_contact_radius / suction_inner_contact_radius - 1))
                      + upper_pressure * ((1 - suction_inner_contact_radius / suction_radius)
                                          / (1 - suction_inner_contact_radius / suction_outer_contact_radius))
                      - (6.0 * velocity * viscosity * density
                         * (suction_outer_contact_radius**2 - suction_inner_contact_radius**2)
                         * (suction_radius - suction_inner_contact_radius)
                         * (suction_outer_contact_radius - suction_radius))
                      / (((displacement+1e-6)**3)
                         * suction_radius * (suction_outer_contact_radius - suction_inner_contact_radius))
                      )
    for i in range(pk.shape[0]):
        if pk[i] <= 2300:
            pk[i] = 2300
            
    plt.plot(suction_radius, pk)
    plt.show()
    f1 = pk * suction_radius
    

    result = integrate.simps(f1, suction_radius)
    return result

In [ ]:
pk = contact_pressure(0.04, np.linspace(0.03644173,0.04,1000), 0.036, 20.0e5, 20.0e5, 1000, 3.5e-4, 0.0, np.pi/2.0, 0.0)
print(pk)
print((18.0e5 + 20.0e5)/2.0*np.pi*(0.04**2-0.036**2)/4.0)
print(18.0e5*np.pi*(0.036**2)/4.0)
print(20.0e5*np.pi*(0.04**2)/4.0)

In [ ]:
(((0.04/2)**2-(0.03644173/2)**2))*np.pi*20e5

In [ ]:
ra = 0.04/2.0
ri = 0.03644173/2.0
pa = 21e5
pf = 20e5
F = np.pi / 2.0 *(ra**2 * pf + ra*ri*(pa-pf)-ri**2 * pa)

In [ ]:
print(F)